In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 한글 폰트 오류 해결
from matplotlib import font_manager, rc 
font_path = "C:\\Users\\이혜림\\Desktop\\Bita5/malgun.ttf" #폰트 파일의 위치
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc("font",family=font_name)

In [79]:
data  = pd.read_csv("정리데이터/prepared(app_users열추가).csv")

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.model_selection import cross_val_score, cross_val_predict

## xgboost 모델 이용
- task
    1. 실제 날씨를 이용한 것과 예보 날씨를 이용한 것의 성능 차이 비교
        - 결과 : 실제 날씨가 좀 더 낮은 mape
    1. 날씨를 각 칼럼으로 넣었을 때와 하나의 값으로 넣었을 때
        - 결과 : 각 지역별 날씨를 평균내서 하나의 칼럼으로 넣었을 때 더 높은 성능
    1. 각 방법별 성능 비교
    1. 임베딩 방법 비교
        - one-hot/labeling 등등등

In [65]:
# 성능 평가 함수 생성
def mape(t, y):
    return 100*(np.abs((y-t)/t).sum())/t.shape[0]

#### 전체 데이터셋을 대상으로 돌리기

In [22]:
"""
# 일 칼럼 추가
data["일"]=pd.to_datetime(data["날짜"]).dt.day

# one-hot-encoding를 위해 int -> 문자열로 변경
data["마더코드"]=data["마더코드"].astype(str)
data["상품코드"]=data["마더코드"].astype(str)

# 카테고리가 무형인 데이터 제거
data = data.drop(data.loc[data["상품군"]=="무형",:].index)
"""
# 데이터셋 변경 완료

In [175]:
# 실제 날씨 칼럼을 이용하는 칼럼 후보
real_columns = ["노출(분)", "마더코드", "상품코드", "상품명", "상품군", "판매단가", "취급액", "요일","시","월" ,"일","요일","holiday",
            '실제_서울_최고기온', '실제_서울_최저기온',
       '실제_서울_강수량(mm)', '실제_서울_평균풍속(m/s)', '실제_수원_최고기온',
       '실제_수원_최저기온', '실제_수원_강수량(mm)', '실제_수원_평균풍속(m/s)', 
       '실제_파주_최고기온', '실제_파주_최저기온', '실제_파주_강수량(mm)', '실제_파주_평균풍속(m/s)',
       'Active Users']

# 예보 날씨 칼럼을 이용하는 칼럼 후보
predict_columns = ["노출(분)", "마더코드", "상품코드", "상품명", "상품군", "판매단가", "취급액", "요일","시","월" ,"일","요일","holiday",
       '서울풍속','예보_서울_강수확률', '예보_서울_강수량', '예보_서울_일최고기온',
       '예보_서울_일최저기온',  '일산풍속','예보_일산_강수확률', '예보_일산_강수량', '예보_일산_일최고기온', '예보_일산_일최저기온',
       '수원풍속','예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온', '예보_수원_일최저기온',
       'Active Users']


In [86]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '날짜',
       '시간', '요일', '시', '주문량', 'holiday', '월', '실제_서울_최고기온', '실제_서울_최저기온',
       '실제_서울_강수량(mm)', '실제_서울_평균풍속(m/s)', '실제_서울_날씨', '실제_수원_최고기온',
       '실제_수원_최저기온', '실제_수원_강수량(mm)', '실제_수원_평균풍속(m/s)', '실제_수원_날씨',
       '실제_파주_최고기온', '실제_파주_최저기온', '실제_파주_강수량(mm)', '실제_파주_평균풍속(m/s)',
       '실제_파주_날씨', '예보_서울_강수확률', '예보_서울_강수량', '예보_서울_하늘상태', '예보_서울_일최고기온',
       '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량', '예보_일산_일최고기온', '예보_일산_일최저기온',
       '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온', '예보_수원_일최저기온', 'Active Users',
       '일', '서울풍속', '수원풍속', '일산풍속'],
      dtype='object')

In [195]:
# 데이터 전처리 함수
def preprocessing(d, columns, weather_merge = False):
    
    # feature 선택하기
    data = d[columns].copy()
    
    # 날씨 merge
    if weather_merge:
        weather = pd.DataFrame()
        if "예보_수원_강수량" in columns:
            for i in range(13,18):
                weather[i]=data.iloc[:,range(i,28,5)].mean(axis=1)
                
        else:
            for i in range(13,17):
                weather[i]=data.iloc[:,range(i,25,4)].mean(axis=1)

    
        data = pd.concat([data[13:-1], weather], axis=1)

    # one-hot-encoding
    one_hot_columns = ["마더코드","상품명","상품군"]
    data = pd.concat([data,pd.get_dummies(data[one_hot_columns])], axis=1) # one-hot-column 생성
    data = data.drop(one_hot_columns+["상품코드"], axis=1) # 원래 column 제거
    
     
    # scaler한 X, y반환
    return StandardScaler().fit_transform(data), d["주문량"]

In [196]:
# 실제 날씨 데이터를 이용하여 예측

X, y = preprocessing(data, real_columns)

# cross_val_score
xg = xgboost.XGBRegressor()
predicted = cross_val_predict(xg, X,y, cv=3)
mape(predicted, y)

[23:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:30:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:30:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


62.43414078090121

In [197]:
# 실제 날씨 데이터 합친 것 이용하여 예측

X, y = preprocessing(data, real_columns, weather_merge = True)

# cross_val_score
xg = xgboost.XGBRegressor()
predicted = cross_val_predict(xg, X,y, cv=3)
mape(predicted, y)

[23:30:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:31:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:31:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


53.86277555774304

In [189]:
# 예보 날씨 데이터를 이용하여 예측

X, y = preprocessing(data, predict_columns)

# cross_val_score
xg = xgboost.XGBRegressor()
predicted = cross_val_predict(xg, X,y, cv=3)
mape(predicted, y)

[23:24:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:24:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:24:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


69.8929197168745

In [190]:
# 예보 날씨 데이터 합친 것 이용하여 예측

X, y = preprocessing(data, predict_columns, weather_merge = True)

# cross_val_score
xg = xgboost.XGBRegressor()
predicted = cross_val_predict(xg, X,y, cv=3)
mape(predicted, y)

[23:24:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:24:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:25:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


62.32732420752197

-> 실제 날씨가 좀 더 성능이 좋음

### 카테고리별로 돌리기

In [182]:
# 데이터 전처리한 것을 카테고리 별로 반환헤 주는 함수
def cate_preprocessing(d, columns, weather_merge = False):
    
    # feature 선택하기
    data = d[columns+["주문량"]].copy()
    
    # 날씨 merge
    if weather_merge:
        weather = pd.DataFrame()
        if "예보_수원_강수량" in columns:
            for i in range(13,18):
                weather[i]=data.iloc[:,range(i,28,5)].mean(axis=1)
                
        else:
            for i in range(13,17):
                weather[i]=data.iloc[:,range(i,25,4)].mean(axis=1)
                
        data = pd.concat([data[13:-1], weather], axis=1)
        
    # one-hot-encoding
    one_hot_columns = ["마더코드","상품명"]
    data = pd.concat([data,pd.get_dummies(data[one_hot_columns])], axis=1) # one-hot-column 생성
    data = data.drop(one_hot_columns+["상품코드"], axis=1) # 원래 column 제거
    
    for c,x in data.groupby(["상품군"]):
        del x["상품군"]
        
        # 카테고리 이름, X(표준화한), y 반환
        yield c, StandardScaler().fit_transform(x.drop(["주문량"], axis=1)), x["주문량"]

#### 각 지역별 날씨를 각 칼럼으로

In [115]:
predict_mapes = {}
for c, X, y in cate_preprocessing(data, predict_columns):
    # cross_val_score
    xg = xgboost.XGBRegressor()
    predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(predicted, y) # mape
    print(c,mape(predicted, y))
    predict_mapes[c]=error

[22:29:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가구 24.158790928404105
[22:29:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가전 42.64402003897606
[22:29:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:41] WARNING: src/objec

In [111]:
real_mape = {}
for c, X, y in cate_preprocessing(data, real_columns):
    # cross_val_score
    xg = xgboost.XGBRegressor()
    real_predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(real_predicted,y) # mape 계산
    print(c,error)
    real_mape[c]=error

[22:28:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가구 24.529066847811784
[22:28:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가전 45.25803866236356
[22:28:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:28] WARNING: src/objec

In [131]:
pd.DataFrame([list(predict_mapes.keys()), list(predict_mapes.values())]).T

,0,1
0,가구,24.1588
1,가전,42.644
2,건강기능,9.60616
3,농수축,3.35152
4,생활용품,64.0906
5,속옷,10.2484
6,의류,68.762
7,이미용,8.25827
8,잡화,78.2617
9,주방,14.0109


In [133]:
pd.DataFrame([list(real_mape.keys()), list(real_mape.values())]).T

,0,1
0,가구,24.5291
1,가전,45.258
2,건강기능,9.0531
3,농수축,3.27598
4,생활용품,41.2862
5,속옷,11.2088
6,의류,17.6616
7,이미용,8.04556
8,잡화,108.716
9,주방,14.9652


#### 각 지역별 날씨를 하나의 칼럼으로

In [185]:
predict_mapes = {}
for c, X, y in cate_preprocessing(data, predict_columns, weather_merge = True):
    # cross_val_score
    xg = xgboost.XGBRegressor()
    predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(predicted, y) # mape
    print(c,mape(predicted, y))
    predict_mapes[c]=error

[23:22:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가구 17.9756995468051
[23:22:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가전 42.28813008264637
[23:22:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:22:35] WARNING: src/objecti

In [186]:
pd.DataFrame([list(predict_mapes.keys()), list(predict_mapes.values())]).T

,0,1
0,가구,17.9757
1,가전,42.2881
2,건강기능,9.60165
3,농수축,3.35786
4,생활용품,56.6023
5,속옷,17.3221
6,의류,18.1466
7,이미용,8.41052
8,잡화,64.8214
9,주방,15.0518


In [183]:
real_mape = {}
for c, X, y in cate_preprocessing(data, real_columns, weather_merge = True):
    # cross_val_score
    xg = xgboost.XGBRegressor()
    real_predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(real_predicted,y) # mape 계산
    print(c,error)
    real_mape[c]=error

[23:20:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가구 18.71071801157092
[23:20:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
가전 47.09375770625223
[23:20:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:44] WARNING: src/object

In [184]:
pd.DataFrame([list(real_mape.keys()), list(real_mape.values())]).T

,0,1
0,가구,18.7107
1,가전,47.0938
2,건강기능,9.03664
3,농수축,3.25862
4,생활용품,38.1329
5,속옷,11.8931
6,의류,17.324
7,이미용,8.23163
8,잡화,87.347
9,주방,14.9652


-> 마찬가지로 실제 날씨가 좀 더 성능이 좋음